In [ ]:
import sympy as sy

import IPython
from IPython.display import display

import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

from scipy.integrate import odeint

def displayH(a1,a2='', a3='', a4='', a5='', a6='', a7='',):
    latex_a1 = sy.latex(a1)
    latex_a2 = sy.latex(a2)
    latex_a3 = sy.latex(a3)
    latex_a4 = sy.latex(a4)
    latex_a5 = sy.latex(a5)
    latex_a6 = sy.latex(a6)
    latex_a7 = sy.latex(a7)
    display( IPython.core.display.Math(latex_a1 + latex_a2 + latex_a3 + latex_a4 + latex_a5 + latex_a6 + latex_a7)  )

$\newpage$

## 1. TRIAD Algorithm

A spacecraft equipped with a star tracker measures the following two vectors in its body frame:

$$
\mathbf{b}_1 = \begin{bmatrix} 0.2500 \\ 0.5177 \\ 0.8311 \end{bmatrix}
$$
$$
\mathbf{b}_2 = \begin{bmatrix} 0.8479 \\ 0.5040 \\ 0.2018 \end{bmatrix}
$$

An onboard star catalog provides the following unit vectors representing the positions of the tracked stars in the Earth-Centered Inertial (ECI) frame:

$$
\mathbf{r}_1 = \begin{bmatrix} 0.5637 \\ 0.3054 \\ 0.7674 \end{bmatrix}
$$
$$
\mathbf{r}_2 = \begin{bmatrix} 0.2569 \\ 0.9337 \\ 0.2495 \end{bmatrix}
$$

Using the TRIAD algorithm, determine the attitude matrix/DCM that transforms vectors from the ECI frame to the spacecraft body frame.

In [70]:
b1 = np.array([0.2500, 0.5177, 0.8311])
b2 = np.array([0.8479, 0.5040, 0.2018])

r1 = np.array([0.5637, 0.3054, 0.7674])
r2 = np.array([0.2569, 0.9337, 0.2495])

def Triad_Estimate(b1,b2,r1,r2):

    bx = np.cross(b1,b2)/np.linalg.norm(np.cross(b1,b2))
    rx = np.cross(r1,r2)/np.linalg.norm(np.cross(r1,r2))

    return np.outer(b1,r1)+np.outer(bx,rx)+np.outer(np.cross(b1,bx),np.cross(r1,rx))

T_ECI_to_B = Triad_Estimate(b1,b2,r1,r2)

displayH(sy.Symbol(r"T_{ECI\rightarrow B}="),sy.Matrix(T_ECI_to_B))

<IPython.core.display.Math object>

$\newpage$

## 2. Quaternion Solution of Wahba's Problem - Davenport's q Method

A spacecraft is equipped with a solid-state star tracker capable of simultaneously tracking multiple stars. It records vector measurements to 30 stars in the spacecraft body frame. These measurements are provided in the Excel file "bvec_Meas.xls", which contains a matrix of size $3 \times 30$. Each column of the matrix corresponds to a vector $\mathbf{b}_i$, where $i = 1, 2, \dots, 30$.

An onboard star catalog provides the unit vectors representing the positions of the tracked stars in the Earth-Centered Inertial (ECI) frame. These vectors are provided in the Excel file "rvec_Refs.xls", which also contains a $3 \times 30$ matrix. The columns of this matrix represent the individual vectors $\mathbf{v}_i$, corresponding to $\mathbf{b}_i$ for $i = 1, 2, \dots, 30$.

Assuming that all measurements are weighted equally, with $a_i = 0.01$, $i = 1, 2, \dots, 30$.

Use the Davenport’s q method, to determine the following:

(a) The estimated optimal quaternion: $\hat{\mathbf{q}}$

(b) The corresponding attitude matrix/DCM that transforms vectors from the ECI frame to the spacecraft body-frame: $\mathbf{A}(\hat{\mathbf{q}})$

The following equations may be useful in the implementation of the Davenport’s q method:

$$
\mathbf{A}(\mathbf{q}) = \begin{bmatrix} 
q_1^2 - q_2^2 - q_3^2 + q_4^2 & 2(q_1q_2 + q_3q_4) & 2(q_1q_3 - q_2q_4) \\
2(q_1q_2 - q_3q_4) & -q_1^2 + q_2^2 - q_3^2 + q_4^2 & 2(q_2q_3 + q_1q_4) \\
2(q_1q_3 + q_2q_4) & 2(q_2q_3 - q_1q_4) & -q_1^2 - q_2^2 + q_3^2 + q_4^2 
\end{bmatrix} \quad \text{... Attitude matrix/DCM}
$$

$$
\mathbf{B} = \sum_{i=1}^{N} a_i \mathbf{b}_i \mathbf{r}_i^T \quad \text{... Attitude profile matrix}
$$

$$
\mathbf{z} = \begin{bmatrix} B_{23} - B_{32} \\ B_{31} - B_{13} \\ B_{12} - B_{21} \end{bmatrix}
$$

$$
\mathbf{K}(\mathbf{B}) = \begin{bmatrix} \mathbf{B} + \mathbf{B}^T - (\operatorname{tr} \mathbf{B}) \mathbf{I}_3 & \mathbf{z} \\ \mathbf{z}^T & \operatorname{tr} \mathbf{B} \end{bmatrix}
$$


In [ ]:
def Quat_to_DCM(q):
    q_1 = q[0]
    q_2 = q[1]
    q_3 = q[2]
    q_4 = q[3]
    q_1q_2 = q_1*q_2
    q_1q_3 = q_1*q_3
    q_3q_4 = q_3*q_4
    q_2q_4 = q_2*q_4
    q_2q_3 = q_2*q_3
    q_1q_4 = q_1*q_4

    return np.array([[q_1**2 - q_2**2 - q_3**2 + q_4**2, 2*(q_1q_2 + q_3q_4), 2*(q_1q_3 - q_2q_4)],
                     [2*(q_1q_2 - q_3q_4), -q_1**2 + q_2**2 - q_3**2 + q_4**2, 2*(q_2q_3 + q_1q_4)],
                     [2*(q_1q_3 + q_2q_4), 2*(q_2q_3 - q_1q_4), -q_1**2 - q_2**2 + q_3**2 + q_4**2]])

In [101]:
bvec_Meas = pd.read_csv("bvec_Meas.csv",header=None).T.to_numpy()
rvec_Refs = pd.read_csv("rvec_Refs.csv",header=None).T.to_numpy()
weights = len(rvec_Refs)*[0.01]

# Attitude Profile Matrix
B = sum([a*np.outer(b,r.T) for a,b,r in zip(weights,bvec_Meas,rvec_Refs)])

z = np.array([B[1,2]-B[2,1],
              B[2,0]-B[0,2],
              B[0,1]-B[1,0]])

K = np.zeros([4,4])
K[:3,:3] = B+B.T-np.trace(B)*np.eye(3)
K[:3,3] = z
K[3,:3] = z
K[3,3] = np.trace(B)
K

array([[ 0.11264821, -0.15545227,  0.02303415, -0.08917601],
       [-0.15545227, -0.11630002, -0.15376081, -0.04072508],
       [ 0.02303415, -0.15376081,  0.15730757,  0.14747669],
       [-0.08917601, -0.04072508,  0.14747669, -0.15365576]])

In [95]:
K[0:2,0:2]

array([[0., 0.],
       [0., 0.]])

In [90]:
A = np.zeros([3,3])
for a,b,r in zip(weights,bvec_Meas,rvec_Refs):
    A += a*np.outer(b,r.T)

A

array([[-0.02050378, -0.00398779,  0.03187962],
       [-0.15146448, -0.13497789, -0.12146841],
       [-0.00884546, -0.0322924 ,  0.00182591]])